In [87]:
# Bibliotecas utilizadas no projeto
import win32com.client as win32
import pandas as pd
from datetime import datetime

In [97]:
# Base da carteira de pedidos
base = pd.read_excel("planilhas/BASE_DASHBOARD.xlsx")

In [89]:
data_atual = datetime.now()

In [90]:
base_dashboard = base[base['DT_ENTREGA'] >= data_atual]

In [91]:
# Base da planilha de e-mails_forn
emails_forn = pd.read_excel("planilhas/emails_forn.xlsx")

In [92]:
# Base da planilha de e-mails_amigao
emails_amigao = pd.read_excel("planilhas/emails_amigao.xlsx")

In [93]:
# Filtrar fornecedores sem agendamento
fornecedores_sem_agendamento = base_dashboard[base_dashboard['DT_AGENDA'].str.startswith('SEM')]

In [94]:
# Filtrar fornecedores com base nas novas regras
fornecedores_validos = base_dashboard[(base_dashboard['Classificacao do pedido'] == 'Original') & 
                            (base_dashboard['LOCAL_ENT'] == 'CD')]

In [95]:
if not fornecedores_sem_agendamento.empty:
    outlook = win32.Dispatch('Outlook.Application')  # Inicializa o Outlook

    # Dicionário para armazenar os pedidos por fornecedor e usuário
    pedidos_por_fornecedor_usuario = {}

    # Contadores para os e-mails enviados e não enviados
    emails_enviados = 0
    emails_nao_enviados = 0

    # Esse laço de repetição navega pelos fornecedores sem agendamento e agrupa os pedidos
    for index, row in fornecedores_sem_agendamento.iterrows():
        nome_fornecedor = row['DESC_FORN']
        dep = row['DEP']
        pedido = row['PEDIDO']
        usuario = row['NOME USUARIO']

        chave = (nome_fornecedor, usuario)

        if chave not in pedidos_por_fornecedor_usuario:
            pedidos_por_fornecedor_usuario[chave] = set()

        # Adiciona o pedido no dicionário, para evitar repetições
        pedidos_por_fornecedor_usuario[chave].add(pedido)

    # Esse laço de repetição navega sobre os fornecedores e usuários e envia os e-mails
    for (nome_fornecedor, usuario), pedidos in pedidos_por_fornecedor_usuario.items():
        # Encontrar informações de e-mails_forn com base no nome_fornecedor
        email_match = emails_forn.loc[emails_forn['nome_fornecedor'] == nome_fornecedor, ['email', 'email_forn_cc']]

        if not email_match.empty:
            email_fornecedor = email_match['email'].values[0]
            cc_fornecedores = email_match['email_forn_cc'].values[0]

            mail = outlook.CreateItem(0)
            mail.Subject = 'URGENTE – PEDIDO SEM AGENDAMENTO NO PORTAL'  # Assunto do email
            mail.To = email_fornecedor

            # Adiciona cópia em CC se houver informações
            if cc_fornecedores:
                mail.CC = cc_fornecedores

            # Adiciona as descrições adicionais no corpo do e-mail
            data_emissao = pd.to_datetime(base.loc[base['DESC_FORN'] == nome_fornecedor, 'DT_EMISSAO'].values[0])

            email_body = f"Prezado Fornecedor: \n\nIdentificamos em nosso sistema o(s) pedido(s) de compra pendente(s) de agendamento com elevado nível de ruptura em nossas unidades e CD’s.\n\n"
            email_body += f"FORNECEDOR: {nome_fornecedor}\n"
            email_body += f"COD_FORN: {base.loc[base['DESC_FORN'] == nome_fornecedor, 'COD_FORN'].values[0]}\n"

            # Add os pedidos sem duplicar no corpo do e-mail
            email_body += "\nPEDIDOS:\n"
            for pedido in pedidos:
                # Encontrar o LEAD TIME correspondente a cada pedido
                lead_time_pedido = base.loc[base['PEDIDO'] == pedido, 'DT_ENTREGA'].values[0] # base['DT_EMISSAO'] >= data_atual)
                emissao_pedido = base.loc[base['PEDIDO'] == pedido, 'DT_EMISSAO'].values[0]
                email_body += f"- {pedido} / LEAD TIME: {pd.to_datetime(lead_time_pedido).strftime('%d-%m-%Y')} / DATA DE EMISSÃO: {pd.to_datetime(lead_time_pedido).strftime('%d-%m-%Y')}\n"

            # Adiciona as demais descrições no corpo do e-mail
            email_body += f"\nDEPARTAMENTO: {base.loc[base['DESC_FORN'] == nome_fornecedor, 'DEP'].values[0]}\n"
            email_body += f"COMPRADOR: {usuario}\n"

            email_body += "\nGentileza agendar na data mais próxima disponível para mitigarmos a ruptura atual. Qualquer dificuldade sinalizar através do e-mail: agendamento@grupoamigao.com\n\n"
            email_body += "Caso o pedido já esteja agendado desconsiderar.\n\n"
            email_body += "No aguardo.\n"

            # Verifica se o "DEP" na base é igual ao "depto" da emails_amigao
            departamento_match = emails_amigao.loc[emails_amigao['depto'] == dep, 'depto'].values

            if len(departamento_match) > 0 and base.loc[base['DESC_FORN'] == nome_fornecedor, 'DEP'].values[0] == departamento_match[0]:
               
                email_usuario_match = emails_amigao.loc[emails_amigao['depto'] == dep, 'email-usuario'].values

                # Adiciona cópia em CC se houver informações
                if len(email_usuario_match) > 0:
                    if mail.CC:
                        mail.CC += ';' + email_usuario_match[0]
                    else:
                        mail.CC = email_usuario_match[0]

                # corpo do email recebe os valores armazenado em email_body
                mail.Body = email_body
                
                # envio de e-mail com tratamento de erro
                try:
                    mail.Send() # envio
                    emails_enviados += 1 # contagem para emails enviados
                except Exception as e:
                    print(f"Erro ao enviar e-mail para {nome_fornecedor} ({usuario}): {e}")
                    emails_nao_enviados += 1 # contagem para emails não enviados
            else:
                print(f"Departamento não corresponde para {nome_fornecedor} ({usuario})")
                emails_nao_enviados += 1
        else:
            print(f"Informações de e-mail não encontradas para {nome_fornecedor} ({usuario})")
            emails_nao_enviados += 1

    print(f"E-mails enviados com sucesso: {emails_enviados}")
    print(f"E-mails não enviados, fornecedores fora da regra: {emails_nao_enviados}")

Departamento não corresponde para Froneri Brasil Distribuidora De Sorv **vera Cruz** (Keze Fabiana Bezerra)
E-mails enviados com sucesso: 6
E-mails não enviados, adicione o email na base 'emails-forn': 1
